<a href="https://colab.research.google.com/github/mrMichalR/Fake_news_detection/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fake news detection


In [ ]:
!pip install ydata_profiling
!pip install torch
!pip install optuna
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.3/352.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.4/455.4 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 37.7 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=88f8c6b704153679b7b02b2cf06350d119fdb39e243107de77461aed92ab6a8a
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: wordcloud
    Found existing installation: wordcloud 1.8.2.2
    

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, sys
import string
import matplotlib.pyplot as plt
from sklearn.utils import resample
import re
import nltk
import torch
import torch.optim as optim
import torch.nn as nn
import optuna
import copy
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
import torch.autograd as autograd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import mean_squared_error
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/week2

Mounted at /content/drive
/content/drive/MyDrive/week2


## Datasets
Get datasets from Kaggle.

1. WELFake <br>
https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification
2. Fake & Real <br>
https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?select=True.csv
3. Fake news net <br>
https://www.kaggle.com/datasets/algord/fake-news



In [ ]:
root = os.getcwd()
path = os.path.join(root, ".csv")
files_list=[]

for path, subdirs, files in os.walk(root):
  for name in files:
    temp_file = os.path.join(path, name)
    print(temp_file)
    files_list.append(temp_file)

# print(files_list)

/content/drive/MyDrive/week2/Fake.csv
/content/drive/MyDrive/week2/True.csv
/content/drive/MyDrive/week2/FakeNewsNet.csv
/content/drive/MyDrive/week2/WELFake_Dataset.csv
/content/drive/MyDrive/week2/output3.html
/content/drive/MyDrive/week2/output1.html
/content/drive/MyDrive/week2/output2.html
/content/drive/MyDrive/week2/ProfileReport1.html
/content/drive/MyDrive/week2/ProfileReport3.html
/content/drive/MyDrive/week2/ProfileReport2.html
/content/drive/MyDrive/week2/df2_downsampled_Fake_&_Real.csv
/content/drive/MyDrive/week2/df1_downsampled.csv
/content/drive/MyDrive/week2/df3_downsampled.csv
/content/drive/MyDrive/week2/df2_downsampled.csv


In [ ]:
df_fake = pd.read_csv(files_list[0])
df_true = pd.read_csv(files_list[1])
df_fake['fake'] = 1
df_true['fake'] = 0

df1=pd.concat([df_fake, df_true], axis=0, ignore_index=False)
print(f'DataFrame of {files_list[0]} and {files_list[1]} file')
df1

DataFrame of /content/drive/MyDrive/week2/Fake.csv and /content/drive/MyDrive/week2/True.csv file


,title,text,subject,date,fake
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0


**ATTENTION !
Due to memory use datasets are sampled to 5000 records each.

In [ ]:
df1 = df1.drop(['subject', 'date','text'], axis=1)
df1 = df1.sample(2500)
df1

,title,fake
17176,Son of Thailand's ex-PM Thaksin charged with m...,0
17011,OBAMA Defends Black Lives Matter But Does He R...,1
7264,Georgia congressman being eyed for top Trump h...,0
10942,Struggling Republican Bush brings out the big ...,0
1720,Ex-Trump aide Manafort's spokesman testifies t...,0
...,...,...
11597,BREAKING #CNNLeak…James O’Keefe’s New Undercov...,1
20130,OOPS! New App Allows Users To Remain ANONYMOUS...,1
20854,Hurricane Irma to move over portions of Virgin...,0
17815,China's Xi looks set to keep right-hand man on...,0


In [ ]:
df2=pd.read_csv(files_list[2])
df2.rename(columns = {'real':'fake'}, inplace = True)
df2 = df2.drop(['news_url', 'source_domain','tweet_num'], axis=1)
df2 = df2.sample(5000)
df2

,title,fake
6060,Vanessa Hudgens joining 'So You Think You Can ...,1
8178,June Thompson Net Worth,1
546,"Ellen DeGeneres Bio, Age, Married, Husband, Da...",0
4773,Chris Stapleton announces he and wife are expe...,1
18149,Celeb Glam Squads Reveal Tips From the Vanity ...,1
...,...,...
962,"Camila Cabello Is the New Face of Guess: Plus,...",1
2367,"Serena Williams, Alexis Ohanian 'Waiting' to L...",1
17600,Tyler Perry’s House Of Payne Is Coming Back to...,1
22432,Savannah Chrisley Splits from Boyfriend Luke K...,1


In [ ]:
df3 = pd.read_csv(files_list[3])
df3.rename(columns = {'label':'fake'}, inplace = True)
df3 = df3.drop(['Unnamed: 0','text'], axis=1)
df3 = df3.sample(2500)
df3

,title,fake
13643,"Hollywood Holiday Habits: Big Games, Small Gif...",0
51512,Taiwan says shut out of U.N. climate talks due...,0
16602,Trump Whines Like A Baby In Defense Of Russia...,1
63468,Chile's Guillier concedes presidency to Pinera,0
48613,"Hillary Clinton Wants a Strong Russia. Wait, w...",1
...,...,...
8329,U.N. urges calm as deadline nears for asylum s...,0
21995,Protest against Bahraini king's visit to Downi...,1
41730,Ex-Miss Finland Just Accused Trump Of Sexual A...,1
67572,Why the latest Patriot Act reform won’t be eno...,0


All 3 in 1 list

In [ ]:
df_list = []
df_list.append(df1)
df_list.append(df2)
df_list.append(df3)

## Data analysis
Analyze the data and perform exploratory data analysis.

1. Check if the dataset is balanced - if not think about methods to acquire balance.
2. Use pandas-profiler


In [ ]:
target='fake'
for number, elem in zip(range(1,4), df_list):
  print(f'Target value counts for df{number}:\n{elem[target].value_counts()}\nSUM:{len(elem)}\n')

Target value counts for df1:
1    1332
0    1168
Name: fake, dtype: int64
SUM:2500

Target value counts for df2:
1    3781
0    1219
Name: fake, dtype: int64
SUM:5000

Target value counts for df3:
1    1296
0    1204
Name: fake, dtype: int64
SUM:2500



In [ ]:
# check duplicates in dataset
for number, elem in zip(range(1,4), df_list):
  print(f'Duplicates for df{number}:\n{elem.duplicated(keep="last").sum()}\n')

Duplicates for df1:
24

Duplicates for df2:
91

Duplicates for df3:
25



In [ ]:
# drop duplicates in dataframes
for number, elem in zip(range(1,4), df_list):
  elem.drop_duplicates(keep="last", inplace=True)
  print(f'Target value counts for df{number}:\n{elem[target].value_counts()}\nSUM:{len(elem)}\n')

Target value counts for df1:
1    1309
0    1167
Name: fake, dtype: int64
SUM:2476

Target value counts for df2:
1    3716
0    1193
Name: fake, dtype: int64
SUM:4909

Target value counts for df3:
1    1271
0    1204
Name: fake, dtype: int64
SUM:2475



In [ ]:
# imbalance check in %
for number, elem in zip(range(1,4), df_list):
  print(f'Target imbalance for df{number}:\n{(elem[target].value_counts()[1]*100/elem[target].value_counts()[0]-100).round(2)} %\n')

Target imbalance for df1:
12.17 %

Target imbalance for df2:
211.48 %

Target imbalance for df3:
5.56 %



**Pandas profiling**

Due to quite big of datasets minimal=True.


Other solution is to take sample 10000 into account.

In [ ]:
# description = "Disclaimer: this profiling report was generated using minimal mode"

# for number, elem in zip(range(1,4), df_list):
#   prof = elem.profile_report(dataset={"description": description}, minimal=True)
#   prof.to_notebook_iframe()
#   prof.to_file(output_file=f'ProfileReport{number}.html')

### Summary
- All 3 datasets are unbalanced

For further investigation:
- Oversample minority class

or

- Undersample majority class

## Data preprocessing
Use preprocessing methods.

1. Think how to handle nulls and nan values

2. Propose other techniques to data preprocessing

In [ ]:
# check for nan values
for number, elem in zip(range(1,4), df_list):
  print(f'Missing values in df{number}:\n{elem.isna().sum()} %\n')

Missing values in df1:
title    0
fake     0
dtype: int64 %

Missing values in df2:
title    0
fake     0
dtype: int64 %

Missing values in df3:
title    1
fake     0
dtype: int64 %



In [ ]:
# drop nan values
for number, elem in zip(range(1,4), df_list):
  elem.dropna(inplace=True)
  print(f'Target value counts for df{number}:\n{elem[target].value_counts()}\nSUM:{len(elem)}\n')

Target value counts for df1:
1    1309
0    1167
Name: fake, dtype: int64
SUM:2476

Target value counts for df2:
1    3716
0    1193
Name: fake, dtype: int64
SUM:4909

Target value counts for df3:
1    1270
0    1204
Name: fake, dtype: int64
SUM:2474



### Undersampling (majority class)

In [ ]:
df1_majority = df1[df1['fake'] == 0]
df1_minority = df1[(df1['fake'] == 1)]

df1_majority_downsampled = resample(df1_majority, replace = True, n_samples = len(df1_minority), random_state = 42)
df1_downsampled = pd.concat([df1_minority, df1_majority_downsampled])
df1_downsampled.to_csv('df1_downsampled.csv', index=False)

df1 = df1_downsampled
df_list[0] = df1_downsampled
print(f'Target value in df1 after udersampling:\n{df_list[0].fake.value_counts()}')


df2_majority = df2[df2['fake'] == 1]
df2_minority = df2[(df2['fake'] == 0)]

df2_majority_downsampled = resample(df2_majority, replace = True, n_samples = len(df2_minority), random_state = 42)
df2_downsampled = pd.concat([df2_minority, df2_majority_downsampled])
df2_downsampled.to_csv('df2_downsampled.csv', index=False)

df2 = df2_downsampled
df_list[1] = df2_downsampled
print(f'Target value in df2 after udersampling:\n{df_list[1].fake.value_counts()}')


df3_majority = df3[df3['fake'] == 0]
df3_minority = df3[(df3['fake'] == 1)]

df3_majority_downsampled = resample(df3_majority, replace = True, n_samples = len(df3_minority), random_state = 42)
df3_downsampled = pd.concat([df3_minority, df3_majority_downsampled])
df3_downsampled.to_csv('df3_downsampled.csv', index=False)

df3 = df3_downsampled
df_list[2] = df3_downsampled
print(f'Target value in df3 after udersampling:\n{df_list[2].fake.value_counts()}')

Target value in df1 after udersampling:
1    1309
0    1309
Name: fake, dtype: int64
Target value in df2 after udersampling:
0    1193
1    1193
Name: fake, dtype: int64
Target value in df3 after udersampling:
1    1270
0    1270
Name: fake, dtype: int64


## Data quality
Check the quality of your data

Source: https://www.geeksforgeeks.org/text-preprocessing-in-python-set-1/?ref=lbp  &  https://www.datacamp.com/tutorial/text-classification-python

1. Convert them to lower or upper case
2. Cut “stopwords”
3. Convert words to their stem or lemma
4. Plan how do you want to divide each sentence? There are a lot of tokenizers?
5. *remove all special characters
6. *remove digits
7. *remove punctuation
8. *remove whitespace

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
lemmatizer = WordNetLemmatizer()

def clean_data(dataframe):
  input_data = list(dataframe)
  corpus = []
  text = list(dataframe)
  for i in range(len(text)):
          # remove all special characters
          tokens = re.sub('[^a-zA-Z]', ' ', text[i])
          # remove digits
          tokens = re.sub(r'\d+', '', tokens)
          # to lower case
          tokens = tokens.lower()
          # tokenize
          tokens = word_tokenize(tokens)  #alternative tokens = tokens.split()
          # remove punctuation
          tokens = [word for word in tokens if word.isalpha()]
          # filter out short tokens (<=1 character)
          tokens = [word for word in tokens if len(word) > 1]
          # remove stopwords
          tokens = [word for word in tokens if word not in stopwords.words('english')]
          # lemmatize
          tokens = [lemmatizer.lemmatize(word) for word in tokens]
          # tokens = [lemmatizer.lemmatize(word, pos ='v') for word in tokens] #lemmatize with provide context i.e. part-of-speech -> pos
          # remove whitespace
          tokens = ' '.join(tokens)

          corpus.append(tokens)

  # assign corpus to data column
  input_data = corpus
  return input_data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
for df in tqdm(df_list):
  df['title'] = clean_data(df['title'])

  0%|          | 0/3 [00:00<?, ?it/s]

## Bag of words
Convert text to vector using classic algorithms

1. Use bag-of-words algorithm
2. Use Bag2Vec algorithm

Sources:

https://learn.microsoft.com/en-us/training/modules/intro-natural-language-processing-pytorch/3-bow-tfidf

WordCloud:

https://www.datacamp.com/tutorial/wordcloud-python


### Bag of words

In [ ]:
vec_cv1 = CountVectorizer(ngram_range=(1,1)) #unigram = 1 word
vec_cv2 = CountVectorizer(ngram_range=(1,1))
vec_cv3 = CountVectorizer(ngram_range=(1,1))
# vec_cv_bi = CountVectorizer(ngram_range=(2,2)) #bi-gram = 2 words

df1_vec = vec_cv1.fit_transform(df1['title']).toarray()
df2_vec = vec_cv2.fit_transform(df2['title']).toarray()
df3_vec = vec_cv3.fit_transform(df3['title']).toarray()

print(f'df1_vec shape is: {df1_vec.shape}')
print(f'df2_vec shape is: {df2_vec.shape}')
print(f'df3_vec shape is: {df3_vec.shape}')

print(f'Type df1_vec: {type(df1_vec)}')
print(f'Type df2_vec: {type(df2_vec)}')
print(f'Type df3_vec: {type(df3_vec)}')

print(f'VOCABULARY 1 (lenght {len(vec_cv1.vocabulary_)})\n{vec_cv1.vocabulary_}')
print(f'VOCABULARY 2 (lenght {len(vec_cv2.vocabulary_)})\n{vec_cv2.vocabulary_}')
print(f'VOCABULARY 3 (lenght {len(vec_cv3.vocabulary_)})\n{vec_cv3.vocabulary_}')

df1_vec shape is: (2618, 5237)
df2_vec shape is: (2386, 5062)
df3_vec shape is: (2540, 5464)
Type df1_vec: <class 'numpy.ndarray'>
Type df2_vec: <class 'numpy.ndarray'>
Type df3_vec: <class 'numpy.ndarray'>
VOCABULARY 1 (lenght 5237)
{'obama': 3188, 'defends': 1222, 'black': 493, 'life': 2720, 'matter': 2878, 'really': 3773, 'want': 5058, 'support': 4562, 'video': 5005, 'updated': 4946, 'america': 155, 'conservative': 994, 'family': 1711, 'experience': 1645, 'shocking': 4244, 'abuse': 15, 'shaming': 4223, 'home': 2215, 'ordered': 3257, 'leftist': 2679, 'da': 1145, 'part': 3342, 'insane': 2388, 'vendetta': 4985, 'boom': 550, 'trump': 4835, 'show': 4256, 'good': 1996, 'business': 659, 'mind': 2967, 'never': 3131, 'let': 2697, 'opportunity': 3244, 'go': 1985, 'waste': 5078, 'online': 3230, 'lib': 2708, 'site': 4296, 'publishes': 3667, 'cell': 751, 'number': 3176, 'revives': 3954, 'keystone': 2555, 'dakota': 1150, 'access': 21, 'pipeline': 3447, 'state': 4464, 'american': 156, 'moving': 30

As extras we can add code to keep words that occurs specific minimum times and reduce the size of voacb.

In [ ]:
# keep tokens/words with a min occurrence
min_occurence = 5
words_to_drop = []


bow1 = pd.DataFrame(df1_vec, columns = vec_cv1.get_feature_names_out())
bow2 = pd.DataFrame(df2_vec, columns = vec_cv2.get_feature_names_out())
bow3 = pd.DataFrame(df3_vec, columns = vec_cv3.get_feature_names_out())


def filter_words(bow_df):
  words_to_drop = []
  for col in bow_df.columns:
    if bow_df[col].sum() < min_occurence:
      words_to_drop.append(col)
  # print(f'Words to drop: {len(words_to_drop)}')
  bow_df_filter = bow_df.drop(words_to_drop, axis=1)
  # print(f'BoW vocab shape after reduction: {bow1_filter.shape}')
  return bow_df_filter


bow1_filter = filter_words(bow1)
bow1_vocab_size = bow1_filter.shape[1]
print(f'BoW 1 shape: {bow1_filter.shape}, length of vocab is: {bow1_vocab_size}')

bow2_filter = filter_words(bow2)
bow2_vocab_size = bow2_filter.shape[1]
print(f'BoW 2 shape: {bow2_filter.shape}, length of vocab is: {bow2_vocab_size}')

bow3_filter = filter_words(bow3)
bow3_vocab_size = bow3_filter.shape[1]
print(f'BoW 3 shape: {bow3_filter.shape}, length of vocab is: {bow3_vocab_size}')

def get_vocab_size(df):
  return df.shape[1]

BoW 1 shape: (2618, 1184), length of vocab is: 1184
BoW 2 shape: (2386, 923), length of vocab is: 923
BoW 3 shape: (2540, 1070), length of vocab is: 1070


### TF-IDF

In [ ]:
# fit & transform
def tfidf_transform(df_features):
  tfidf = TfidfVectorizer()
  df = tfidf.fit_transform(df_features)
  vocab_size = len(tfidf.vocabulary_)
  print(f'Shape is: {df.shape}')
  print(f'Type: {type(df)}')
  print(f'VOCABULARY (lenght {vocab_size})\n{tfidf.vocabulary_}')
  # print(tfidf1.vocabulary_)
  # print(tfidf.idf_)
  return df, vocab_size

tfidf1, tfidf1_vocab_size = tfidf_transform(df1['title'])
tfidf2, tfidf2_vocab_size = tfidf_transform(df2['title'])
tfidf3, tfidf3_vocab_size = tfidf_transform(df3['title'])

tfidf1 = tfidf1.toarray()
tfidf2 = tfidf2.toarray()
tfidf3 = tfidf3.toarray()

Shape is: (2618, 5237)
Type: <class 'scipy.sparse._csr.csr_matrix'>
VOCABULARY (lenght 5237)
{'obama': 3188, 'defends': 1222, 'black': 493, 'life': 2720, 'matter': 2878, 'really': 3773, 'want': 5058, 'support': 4562, 'video': 5005, 'updated': 4946, 'america': 155, 'conservative': 994, 'family': 1711, 'experience': 1645, 'shocking': 4244, 'abuse': 15, 'shaming': 4223, 'home': 2215, 'ordered': 3257, 'leftist': 2679, 'da': 1145, 'part': 3342, 'insane': 2388, 'vendetta': 4985, 'boom': 550, 'trump': 4835, 'show': 4256, 'good': 1996, 'business': 659, 'mind': 2967, 'never': 3131, 'let': 2697, 'opportunity': 3244, 'go': 1985, 'waste': 5078, 'online': 3230, 'lib': 2708, 'site': 4296, 'publishes': 3667, 'cell': 751, 'number': 3176, 'revives': 3954, 'keystone': 2555, 'dakota': 1150, 'access': 21, 'pipeline': 3447, 'state': 4464, 'american': 156, 'moving': 3060, 'build': 633, 'helipad': 2167, 'mar': 2853, 'lago': 2613, 'avoid': 313, 'protester': 3648, 'golfing': 1993, 'weekend': 5101, 'whoa': 5128

## Train, validation and test
Divide prepared dataset into three subsets.

Implement method to division dataset into training, validation and testing subsets.

Propose different divisions (in percentage):
1. 80/10/10 (tested in this notebook)
2. 70/15/15
3. 60/20/20

In [ ]:
# Create Feature and Label sets

# BoW
X1_bow = bow1_filter
y1_bow = df1['fake']

X2_bow = bow2_filter
y2_bow = df2['fake']

X3_bow = bow3_filter
y3_bow = df3['fake']


# TFIDF
X1_tfidf = tfidf1
y1_tfidf = df1['fake']

X2_tfidf = tfidf2
y2_tfidf = df2['fake']

X3_tfidf = tfidf3
y3_tfidf = df3['fake']


# train, val & test split (80% train, 10% val, 10% test)

#BOW split
#df1
X_train1, X_temp1, y_train1, y_temp1 = train_test_split(X1_bow, y1_bow, test_size=0.2, random_state=42)
X_val1, X_test1, y_val1, y_test1 = train_test_split(X_temp1, y_temp1, test_size=0.5, random_state=42)

#df2
X_train2, X_temp2, y_train2, y_temp2 = train_test_split(X2_bow, y2_bow, test_size=0.2, random_state=42)
X_val2, X_test2, y_val2, y_test2 = train_test_split(X_temp2, y_temp2, test_size=0.5, random_state=42)

#df3
X_train3, X_temp3, y_train3, y_temp3 = train_test_split(X3_bow, y3_bow, test_size=0.2, random_state=42)
X_val3, X_test3, y_val3, y_test3 = train_test_split(X_temp3, y_temp3, test_size=0.5, random_state=42)

print('Dataset 1 BOW:')
print('X train:', X_train1.shape,'y train: ', y_train1.shape)
print('X val: ', X_val1.shape,'y val: ', y_val1.shape)
print('X test:', X_test1.shape,'y test: ', y_test1.shape)
print('\n')

print('Dataset 2 BOW:')
print('X train:', X_train2.shape,'y train: ', y_train2.shape)
print('X val: ', X_val2.shape,'y val: ', y_val2.shape)
print('X test:', X_test2.shape,'y test: ', y_test2.shape)
print('\n')


print('Dataset 3 BOW:')
print('X train:', X_train3.shape,'y train: ', y_train3.shape)
print('X val: ', X_val3.shape,'y val: ', y_val3.shape)
print('X test:', X_test3.shape,'y test: ', y_test3.shape)
print('\n')

#TFIDF split
#df1
X_train1_tfidf, X_temp1_tfidf, y_train1_tfidf, y_temp1_tfidf = train_test_split(X1_tfidf, y1_tfidf, test_size=0.2, random_state=42)
X_val1_tfidf, X_test1_tfidf, y_val1_tfidf, y_test1_tfidf = train_test_split(X_temp1_tfidf, y_temp1_tfidf, test_size=0.5, random_state=42)

# #df2
X_train2_tfidf, X_temp2_tfidf, y_train2_tfidf, y_temp2_tfidf = train_test_split(X2_tfidf, y2_tfidf, test_size=0.2, random_state=42)
X_val2_tfidf, X_test2_tfidf, y_val2_tfidf, y_test2_tfidf = train_test_split(X_temp2_tfidf, y_temp2_tfidf, test_size=0.5, random_state=42)

# #df3
X_train3_tfidf, X_temp3_tfidf, y_train3_tfidf, y_temp3_tfidf = train_test_split(X3_tfidf, y3_tfidf, test_size=0.2, random_state=42)
X_val3_tfidf, X_test3_tfidf, y_val3_tfidf, y_test3_tfidf = train_test_split(X_temp3_tfidf, y_temp3_tfidf, test_size=0.5, random_state=42)

print('Dataset 1 TFIDF:')
print('X train:', X_train1_tfidf.shape,'y train: ', y_train1_tfidf.shape)
print('X val: ', X_val1_tfidf.shape,'y val: ', y_val1_tfidf.shape)
print('X test:', X_test1_tfidf.shape,'y test: ', y_test1_tfidf.shape)
print('\n')

print('Dataset 2 TFIDF:')
print('X train:', X_train2_tfidf.shape,'y train: ', y_train2_tfidf.shape)
print('X val: ', X_val2_tfidf.shape,'y val: ', y_val2_tfidf.shape)
print('X test:', X_test2_tfidf.shape,'y test: ', y_test2_tfidf.shape)
print('\n')

print('Dataset 3 TFIDF:')
print('X train:', X_train3_tfidf.shape,'y train: ', y_train3_tfidf.shape)
print('X val: ', X_val3_tfidf.shape,'y val: ', y_val3_tfidf.shape)
print('X test:', X_test3_tfidf.shape,'y test: ', y_test3_tfidf.shape)
print('\n')

Dataset 1 BOW:
X train: (2094, 1184) y train:  (2094,)
X val:  (262, 1184) y val:  (262,)
X test: (262, 1184) y test:  (262,)


Dataset 2 BOW:
X train: (1908, 923) y train:  (1908,)
X val:  (239, 923) y val:  (239,)
X test: (239, 923) y test:  (239,)


Dataset 3 BOW:
X train: (2032, 1070) y train:  (2032,)
X val:  (254, 1070) y val:  (254,)
X test: (254, 1070) y test:  (254,)


Dataset 1 TFIDF:
X train: (2094, 5237) y train:  (2094,)
X val:  (262, 5237) y val:  (262,)
X test: (262, 5237) y test:  (262,)


Dataset 2 TFIDF:
X train: (1908, 5062) y train:  (1908,)
X val:  (239, 5062) y val:  (239,)
X test: (239, 5062) y test:  (239,)


Dataset 3 TFIDF:
X train: (2032, 5464) y train:  (2032,)
X val:  (254, 5464) y val:  (254,)
X test: (254, 5464) y test:  (254,)




## Logistic regression
Use PyTorch library.

1. Train logistic regression model using prepared training dataset
2. Test model using testing dataset
3. Use Optuna to find the best hyperparameters the for logistic regression.
4. Evaluate model using k-stratified fold Cross Validation using train, test and validation part of the dataset

Create tensors

In [ ]:
# BoW tensors for all dataset
X1_bow_tens = torch.tensor(X1_bow.values, dtype=torch.float32)
y1_bow_tens = torch.tensor(y1_bow.values, dtype=torch.float32).reshape(-1,1)
print(X1_bow_tens.shape,y1_bow_tens.shape)

X2_bow_tens = torch.tensor(X2_bow.values, dtype=torch.float32)
y2_bow_tens = torch.tensor(y2_bow.values, dtype=torch.float32).reshape(-1,1)
print(X2_bow_tens.shape,y2_bow_tens.shape)

X3_bow_tens = torch.tensor(X3_bow.values, dtype=torch.float32)
y3_bow_tens = torch.tensor(y3_bow.values, dtype=torch.float32).reshape(-1,1)
print(X2_bow_tens.shape,y3_bow_tens.shape)


# TFIDF tensors for all dataset
X1_tfidf_tens = torch.tensor(X1_tfidf, dtype=torch.float32)
y1_tfidf_tens = torch.tensor(y1_tfidf.values, dtype=torch.float32).reshape(-1,1)
print(X1_tfidf_tens.shape,y1_tfidf_tens.shape)

X2_tfidf_tens = torch.tensor(X2_tfidf, dtype=torch.float32)
y2_tfidf_tens = torch.tensor(y2_tfidf.values, dtype=torch.float32).reshape(-1,1)
print(X2_tfidf_tens.shape,y2_tfidf_tens.shape)

X3_tfidf_tens = torch.tensor(X3_tfidf, dtype=torch.float32)
y3_tfidf_tens = torch.tensor(y3_tfidf.values, dtype=torch.float32).reshape(-1,1)
print(X3_tfidf_tens.shape,y3_tfidf_tens.shape)

torch.Size([2618, 1184]) torch.Size([2618, 1])
torch.Size([2386, 923]) torch.Size([2386, 1])
torch.Size([2386, 923]) torch.Size([2540, 1])
torch.Size([2618, 5237]) torch.Size([2618, 1])
torch.Size([2386, 5062]) torch.Size([2386, 1])
torch.Size([2540, 5464]) torch.Size([2540, 1])


In [ ]:
# BOW train tensors
X1_tens_train = torch.tensor(X_train1.values, dtype=torch.float32)
y1_tens_train = torch.tensor(y_train1.values, dtype=torch.float32).reshape(-1,1)
print('BOW Train sets:\n',X1_tens_train.shape,y1_tens_train.shape)

X2_tens_train = torch.tensor(X_train2.values, dtype=torch.float32)
y2_tens_train = torch.tensor(y_train2.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_train.shape,y2_tens_train.shape)

X3_tens_train = torch.tensor(X_train3.values, dtype=torch.float32)
y3_tens_train = torch.tensor(y_train3.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_train.shape,y3_tens_train.shape)

# BOW val tensors
X1_tens_val = torch.tensor(X_val1.values, dtype=torch.float32)
y1_tens_val = torch.tensor(y_val1.values, dtype=torch.float32).reshape(-1,1)
print('BOW Val sets:\n',X1_tens_val.shape,y1_tens_val.shape)

X2_tens_val = torch.tensor(X_val2.values, dtype=torch.float32)
y2_tens_val = torch.tensor(y_val2.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_val.shape,y2_tens_val.shape)

X3_tens_val = torch.tensor(X_val3.values, dtype=torch.float32)
y3_tens_val = torch.tensor(y_val3.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_val.shape,y3_tens_val.shape)

# BOW test tensors
X1_tens_test = torch.tensor(X_test1.values, dtype=torch.float32)
y1_tens_test = torch.tensor(y_test1.values, dtype=torch.float32).reshape(-1,1)
print('BOW Test sets:\n',X1_tens_test.shape,y1_tens_test.shape)

X2_tens_test = torch.tensor(X_test2.values, dtype=torch.float32)
y2_tens_test = torch.tensor(y_test2.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_test.shape,y2_tens_test.shape)

X3_tens_test = torch.tensor(X_test3.values, dtype=torch.float32)
y3_tens_test = torch.tensor(y_test3.values, dtype=torch.float32).reshape(-1,1)
print(X1_tens_test.shape,y1_tens_test.shape)



# TFIDF train tensors
X1_tens_train_tfidf = torch.tensor(X_train1_tfidf, dtype=torch.float32)
# X2_tens2_tfidf = torch.from_numpy(X_train2).type(torch.float)
y1_tens_train_tfidf = torch.tensor(y_train1_tfidf.values, dtype=torch.float32).reshape(-1,1)
print('TFIDF train:\n',X1_tens_train_tfidf.shape,y1_tens_train_tfidf.shape)

X2_tens_train_tfidf = torch.tensor(X_train2_tfidf, dtype=torch.float32)
y2_tens_train_tfidf = torch.tensor(y_train2_tfidf.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_train_tfidf.shape,y2_tens_train_tfidf.shape)

X3_tens_train_tfidf = torch.tensor(X_train3_tfidf, dtype=torch.float32)
y3_tens_train_tfidf = torch.tensor(y_train3_tfidf.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_train_tfidf.shape,y3_tens_train_tfidf.shape)

# TFIDF val tensors
X1_tens_val_tfidf = torch.tensor(X_val1_tfidf, dtype=torch.float32)
# X2_tens2_tfidf = torch.from_numpy(X_val2).type(torch.float)
y1_tens_val_tfidf = torch.tensor(y_val1_tfidf.values, dtype=torch.float32).reshape(-1,1)
print('TFIDF val:\n',X1_tens_val_tfidf.shape,y1_tens_val_tfidf.shape)

X2_tens_val_tfidf = torch.tensor(X_val2_tfidf, dtype=torch.float32)
y2_tens_val_tfidf = torch.tensor(y_val2_tfidf.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_val_tfidf.shape,y2_tens_val_tfidf.shape)

X3_tens_val_tfidf = torch.tensor(X_val3_tfidf, dtype=torch.float32)
y3_tens_val_tfidf = torch.tensor(y_val3_tfidf.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_val_tfidf.shape,y3_tens_val_tfidf.shape)

# TFIDF test tensors
X1_tens_test_tfidf = torch.tensor(X_test1_tfidf, dtype=torch.float32)
# X2_tens2_tfidf = torch.from_numpy(X_train2).type(torch.float)
y1_tens_test_tfidf = torch.tensor(y_test1_tfidf.values, dtype=torch.float32).reshape(-1,1)
print('TFIDF test:\n',X1_tens_test_tfidf.shape,y1_tens_test_tfidf.shape)

X2_tens_test_tfidf = torch.tensor(X_test2_tfidf, dtype=torch.float32)
y2_tens_test_tfidf = torch.tensor(y_test2_tfidf.values, dtype=torch.float32).reshape(-1,1)
print(X2_tens_test_tfidf.shape,y2_tens_test_tfidf.shape)

X3_tens_test_tfidf = torch.tensor(X_test3_tfidf, dtype=torch.float32)
y3_tens_test_tfidf = torch.tensor(y_test3_tfidf.values, dtype=torch.float32).reshape(-1,1)
print(X3_tens_test_tfidf.shape,y3_tens_test_tfidf.shape)

BOW Train sets:
 torch.Size([2094, 1184]) torch.Size([2094, 1])
torch.Size([1908, 923]) torch.Size([1908, 1])
torch.Size([2032, 1070]) torch.Size([2032, 1])
BOW Val sets:
 torch.Size([262, 1184]) torch.Size([262, 1])
torch.Size([239, 923]) torch.Size([239, 1])
torch.Size([254, 1070]) torch.Size([254, 1])
BOW Test sets:
 torch.Size([262, 1184]) torch.Size([262, 1])
torch.Size([239, 923]) torch.Size([239, 1])
torch.Size([262, 1184]) torch.Size([262, 1])
TFIDF train:
 torch.Size([2094, 5237]) torch.Size([2094, 1])
torch.Size([1908, 5062]) torch.Size([1908, 1])
torch.Size([2032, 5464]) torch.Size([2032, 1])
TFIDF val:
 torch.Size([262, 5237]) torch.Size([262, 1])
torch.Size([239, 5062]) torch.Size([239, 1])
torch.Size([254, 5464]) torch.Size([254, 1])
TFIDF test:
 torch.Size([262, 5237]) torch.Size([262, 1])
torch.Size([239, 5062]) torch.Size([239, 1])
torch.Size([254, 5464]) torch.Size([254, 1])


In [ ]:
# build custom module/net Logistic regression
class LogisticRegression(torch.nn.Module):
    # build the constructor
    def __init__(self, vocab_size, num_labels):
        super().__init__()
        self.linear = torch.nn.Linear(vocab_size, num_labels)
    # make predictions
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [ ]:
def model_train(model, X_train, y_train, X_val, y_val):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = optim.SGD(model.parameters(), lr=0.01)

    n_epochs = 10   # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())

    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

Evaluate model on each df test subset

In [ ]:
# create model, train, and get accuracy
model1 = LogisticRegression(bow1_vocab_size,1)
acc1 = model_train(model1, X1_tens_train, y1_tens_train, X1_tens_val, y1_tens_val)
print("BOW df1\n",'Accuracy (LogReg): %.2f' % acc1)


# evaluate the model on the test set
model1.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred1 = model1(X1_tens_test[i:i+1])
    print(f"{y_pred1[0].numpy()} " +
      f"(expected {y1_tens_test[i].numpy()})")

BOW df1
 Accuracy (LogReg): 0.83
[0.40036616] (expected [0.])
[0.38600034] (expected [0.])
[0.5444438] (expected [1.])
[0.42703295] (expected [0.])
[0.42258814] (expected [0.])


In [ ]:
# create model, train, and get accuracy
model2 = LogisticRegression(bow2_vocab_size,1)
acc2 = model_train(model2, X2_tens_train, y2_tens_train, X2_tens_val, y2_tens_val)
print("BOW df2\n",'Accuracy (LogReg): %.2f' % acc2)


# evaluate the model on the test set
model2.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred2 = model2(X2_tens_test[i:i+1])
    print(f"{y_pred2[0].numpy()} " +
      f"(expected {y2_tens_test[i].numpy()})")

BOW df2
 Accuracy (LogReg): 0.71
[0.54540294] (expected [1.])
[0.5508097] (expected [1.])
[0.54348] (expected [0.])
[0.4955767] (expected [0.])
[0.52054876] (expected [1.])


In [ ]:
# create model, train, and get accuracy
model3 = LogisticRegression(bow3_vocab_size,1)
acc3 = model_train(model3, X3_tens_train, y3_tens_train, X3_tens_val, y3_tens_val)
print("BOW df3\n",'Accuracy (LogReg): %.2f' % acc3)


# evaluate the model on the test set
model3.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred3 = model3(X3_tens_test[i:i+1])
    print(f"{y_pred3[0].numpy()} " +
      f"(expected {y3_tens_test[i].numpy()})")

BOW df3
 Accuracy (LogReg): 0.80
[0.31962997] (expected [0.])
[0.47888017] (expected [0.])
[0.5913829] (expected [1.])
[0.5673602] (expected [1.])
[0.46168706] (expected [0.])


In [ ]:
# create model, train, and get accuracy
model4 = LogisticRegression(get_vocab_size(X1_tens_train_tfidf),1)
acc4 = model_train(model4, X1_tens_train_tfidf, y1_tens_train_tfidf, X1_tens_val_tfidf, y1_tens_val_tfidf)
print("TFIDF df1\n",'Accuracy (LogReg): %.2f' % acc4)


# evaluate the model on the test set
model4.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred4 = model4(X1_tens_test_tfidf[i:i+1])
    print(f"{y_pred4[0].numpy()} " +
      f"(expected {y1_tens_test_tfidf[i].numpy()})")

TFIDF df1
 Accuracy (LogReg): 0.85
[0.4965447] (expected [0.])
[0.4944224] (expected [0.])
[0.505687] (expected [1.])
[0.49815083] (expected [0.])
[0.4937417] (expected [0.])


In [ ]:
# create model, train, and get accuracy
model5 = LogisticRegression(get_vocab_size(X2_tens_train_tfidf),1)
acc5 = model_train(model5, X2_tens_train_tfidf, y2_tens_train_tfidf, X2_tens_val_tfidf, y2_tens_val_tfidf)
print("TFIDF df2\n",'Accuracy (LogReg): %.2f' % acc5)


# evaluate the model on the test set
model5.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred5 = model5(X2_tens_test_tfidf[i:i+1])
    print(f"{y_pred5[0].numpy()} " +
      f"(expected {y2_tens_test_tfidf[i].numpy()})")

TFIDF df2
 Accuracy (LogReg): 0.67
[0.5092374] (expected [1.])
[0.5115051] (expected [1.])
[0.5062474] (expected [0.])
[0.5060356] (expected [0.])
[0.50337833] (expected [1.])


In [ ]:
# create model, train, and get accuracy
model6 = LogisticRegression(get_vocab_size(X3_tens_train_tfidf),1)
acc6 = model_train(model6, X3_tens_train_tfidf, y3_tens_train_tfidf, X3_tens_val_tfidf, y3_tens_val_tfidf)
print("TFIDF df3\n",'Accuracy (LogReg): %.2f' % acc6)


# evaluate the model on the test set
model6.eval()
with torch.no_grad():
  # Test out inference with 5 samples
  for i in range(5):
    y_pred6 = model6(X3_tens_test_tfidf[i:i+1])
    print(f"{y_pred6[0].numpy()} " +
      f"(expected {y3_tens_test_tfidf[i].numpy()})")

TFIDF df3
 Accuracy (LogReg): 0.76
[0.4869378] (expected [0.])
[0.4995272] (expected [0.])
[0.5073697] (expected [1.])
[0.5094066] (expected [1.])
[0.49771306] (expected [0.])


### Optuna

In [ ]:
def model_train_optuna(model, X_train, y_train, X_val, y_val, param):
    # loss function and optimizer
    loss_fn = nn.BCELoss()  # binary cross entropy
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    n_epochs = 10   # number of epochs to run
    batch_size = 10  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(n_epochs):
        model.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred = model(X_val)
        acc = (y_pred.round() == y_val).float().mean()
        acc = float(acc)
        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())

    # restore model and return best accuracy
    model.load_state_dict(best_weights)
    return best_acc

In [ ]:
# objective class with specific arguments
class Objective:
    def __init__(self, X_train, y_train, X_test, y_test, vocab_size):
        # Hold this implementation specific arguments as the fields of the class.
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.vocab_size = vocab_size

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical('optimizer', ["Adam", "SGD", "RMSprop"]),
              'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64]),
              'epochs': trial.suggest_int('epochs', 10, 50)
              }
        model = LogisticRegression(self.vocab_size, 1)
        accuracy = model_train_optuna(model, self.X_train, self.y_train, self.X_test, self.y_test, params)
        return accuracy

In [ ]:
# Execute an optimization by using an `Objective` instance.

# BOW for df1
study_bow1 = optuna.create_study()
study_bow1.optimize(Objective(X1_tens_train, y1_tens_train, X1_tens_test, y1_tens_test, get_vocab_size(X1_tens_train)), n_trials=10)
best_trial_bow1 = study_bow1.best_trial

for key, value in best_trial_bow1.params.items():
    print("{}: {}".format(key, value))

[I 2023-05-24 21:18:56,611] A new study created in memory with name: no-name-03c4d61a-8e1a-4d0b-8976-433f96a73bf9
<ipython-input-35-8c98eff8b77e>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[I 2023-05-24 21:18:57,928] Trial 0 finished with value: 0.9160305261611938 and parameters: {'learning_rate': 0.002223267484741923, 'optimizer': 'Adam', 'batch_size': 32, 'epochs': 41}. Best is trial 0 with value: 0.9160305261611938.
[I 2023-05-24 21:18:59,234] Trial 1 finished with value: 0.9236640930175781 and parameters: {'learning_rate': 0.03060725099332623, 'optimizer': 'Adam', 'batch_size': 16, 'epochs': 22}. Best is trial 0 with value: 0.9160305261611938.
[I 2023-05-24 21:19:00,158] Trial 2 finished with value: 0.9045801758766174 and parameters:

learning_rate: 5.2198075091475116e-05
optimizer: Adam
batch_size: 32
epochs: 18


In [ ]:
# BOW for df2
study_bow2 = optuna.create_study()
study_bow2.optimize(Objective(X2_tens_train, y2_tens_train, X2_tens_test, y2_tens_test, get_vocab_size(X2_tens_train)), n_trials=10)
best_trial_bow2 = study_bow2.best_trial

for key, value in best_trial_bow2.params.items():
    print("{}: {}".format(key, value))

[I 2023-05-24 21:19:08,031] A new study created in memory with name: no-name-b33b9d35-66b9-4ac7-805a-4db6d84edd0a
<ipython-input-35-8c98eff8b77e>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[I 2023-05-24 21:19:09,714] Trial 0 finished with value: 0.7280334830284119 and parameters: {'learning_rate': 0.0032507670540180746, 'optimizer': 'Adam', 'batch_size': 32, 'epochs': 26}. Best is trial 0 with value: 0.7280334830284119.
[I 2023-05-24 21:19:11,485] Trial 1 finished with value: 0.7280334830284119 and parameters: {'learning_rate': 0.0002944429851430488, 'optimizer': 'Adam', 'batch_size': 16, 'epochs': 13}. Best is trial 0 with value: 0.7280334830284119.
[I 2023-05-24 21:19:12,335] Trial 2 finished with value: 0.6987447738647461 and paramete

learning_rate: 1.5130566581990684e-05
optimizer: SGD
batch_size: 64
epochs: 31


In [ ]:
# BOW for df3
study_bow3 = optuna.create_study()
study_bow3.optimize(Objective(X3_tens_train, y3_tens_train, X3_tens_test, y3_tens_test, get_vocab_size(X3_tens_train)), n_trials=10)
best_trial_bow3 = study_bow3.best_trial

for key, value in best_trial_bow3.params.items():
    print("{}: {}".format(key, value))

[I 2023-05-24 21:19:18,872] A new study created in memory with name: no-name-4c68dcc1-b288-4ea2-8801-acfda1081b35
<ipython-input-35-8c98eff8b77e>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[I 2023-05-24 21:19:20,124] Trial 0 finished with value: 0.7992125749588013 and parameters: {'learning_rate': 8.435950617654025e-05, 'optimizer': 'Adam', 'batch_size': 32, 'epochs': 50}. Best is trial 0 with value: 0.7992125749588013.
[I 2023-05-24 21:19:21,342] Trial 1 finished with value: 0.7716535329818726 and parameters: {'learning_rate': 6.646040943107781e-05, 'optimizer': 'Adam', 'batch_size': 16, 'epochs': 26}. Best is trial 1 with value: 0.7716535329818726.
[I 2023-05-24 21:19:22,612] Trial 2 finished with value: 0.8385826945304871 and paramete

learning_rate: 0.00039090648642125755
optimizer: SGD
batch_size: 32
epochs: 36


In [ ]:
#TFIDF for df1
study_tfidf1 = optuna.create_study()
study_tfidf1.optimize(Objective(X1_tens_train_tfidf, y1_tens_train_tfidf, X1_tens_test_tfidf, y1_tens_test_tfidf, tfidf1_vocab_size), n_trials=10)
best_trial_tfidf1 = study_tfidf1.best_trial

for key, value in best_trial_tfidf1.params.items():
    print("{}: {}".format(key, value))

[I 2023-05-24 21:19:31,355] A new study created in memory with name: no-name-bde90631-dc25-4510-81d7-f09a71300b51
<ipython-input-35-8c98eff8b77e>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[I 2023-05-24 21:19:32,792] Trial 0 finished with value: 0.9389312863349915 and parameters: {'learning_rate': 0.043915281634564526, 'optimizer': 'Adam', 'batch_size': 16, 'epochs': 37}. Best is trial 0 with value: 0.9389312863349915.
[I 2023-05-24 21:19:34,189] Trial 1 finished with value: 0.9465649127960205 and parameters: {'learning_rate': 0.013049003098027354, 'optimizer': 'Adam', 'batch_size': 32, 'epochs': 15}. Best is trial 0 with value: 0.9389312863349915.
[I 2023-05-24 21:19:35,406] Trial 2 finished with value: 0.8816794157028198 and parameters

learning_rate: 0.0002285373866022774
optimizer: SGD
batch_size: 32
epochs: 25


In [ ]:
#TFIDF for df2
study_tfidf2 = optuna.create_study()
study_tfidf2.optimize(Objective(X2_tens_train_tfidf, y2_tens_train_tfidf, X2_tens_test_tfidf, y2_tens_test_tfidf, tfidf2_vocab_size), n_trials=10)
best_trial_tfidf2 = study_tfidf2.best_trial

for key, value in best_trial_tfidf2.params.items():
    print("{}: {}".format(key, value))


[I 2023-05-24 21:19:45,907] A new study created in memory with name: no-name-ffb5ab99-4d85-46c2-861e-56e77233f90f
<ipython-input-35-8c98eff8b77e>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[I 2023-05-24 21:19:46,835] Trial 0 finished with value: 0.589958131313324 and parameters: {'learning_rate': 0.003882372609750014, 'optimizer': 'SGD', 'batch_size': 32, 'epochs': 48}. Best is trial 0 with value: 0.589958131313324.
[I 2023-05-24 21:19:47,899] Trial 1 finished with value: 0.73221755027771 and parameters: {'learning_rate': 0.002913102566594277, 'optimizer': 'RMSprop', 'batch_size': 64, 'epochs': 46}. Best is trial 0 with value: 0.589958131313324.
[I 2023-05-24 21:19:49,230] Trial 2 finished with value: 0.73221755027771 and parameters: {'l

learning_rate: 0.00012735470749111524
optimizer: SGD
batch_size: 16
epochs: 31


In [ ]:
#TFIDF for df3
study_tfidf3 = optuna.create_study()
study_tfidf3.optimize(Objective(X3_tens_train_tfidf, y3_tens_train_tfidf, X3_tens_test_tfidf, y3_tens_test_tfidf, tfidf3_vocab_size), n_trials=10)
best_trial_tfidf3 = study_tfidf3.best_trial

for key, value in best_trial_tfidf3.params.items():
    print("{}: {}".format(key, value))

[I 2023-05-24 21:19:57,955] A new study created in memory with name: no-name-d38d0def-934a-4bb7-89df-e2fc5bb84a4b
<ipython-input-35-8c98eff8b77e>:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
[I 2023-05-24 21:20:00,175] Trial 0 finished with value: 0.874015748500824 and parameters: {'learning_rate': 0.01821692826420452, 'optimizer': 'RMSprop', 'batch_size': 16, 'epochs': 20}. Best is trial 0 with value: 0.874015748500824.
[I 2023-05-24 21:20:01,543] Trial 1 finished with value: 0.6023622155189514 and parameters: {'learning_rate': 0.0012173455352747873, 'optimizer': 'SGD', 'batch_size': 64, 'epochs': 12}. Best is trial 1 with value: 0.6023622155189514.
[I 2023-05-24 21:20:03,299] Trial 2 finished with value: 0.7086614370346069 and parameters

learning_rate: 0.0012173455352747873
optimizer: SGD
batch_size: 64
epochs: 12


### Best parameters evaluation

In [ ]:
def model_train_best(model, X_train, y_train, X_test, y_test, best_trial, vocab_size):
    # train the model on the full training set with the best hyperparameters
    lr = best_trial.params['learning_rate']
    optimizer_best = best_trial.params['optimizer']
    batch_size = best_trial.params['batch_size']
    epochs = best_trial.params['epochs']

    optimizer_temp = 'optim.'+optimizer_best+'(model.parameters(), lr=lr)'
    optimizer = eval(optimizer_temp)

    loss_fn = nn.BCELoss()  # binary cross entropy

    # batch_size = batch_size  # size of each batch
    batch_start = torch.arange(0, len(X_train), batch_size)
    model = LogisticRegression(vocab_size, 1)

    # Hold the best model
    best_acc = - np.inf   # init to negative infinity
    best_weights = None

    for epoch in range(epochs):
        model.train()
        with tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:
                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]
                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)
                # backward pass
                optimizer.zero_grad()
                loss.backward()
                # update weights
                optimizer.step()
                # print progress
                acc = (y_pred.round() == y_batch).float().mean()
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc))

    # evaluate accuracy at end of epochs
    model.eval()
    y_pred = model(X_test)
    acc = (y_pred.round() == y_test).float().mean()
    acc = float(acc)
    return acc

Retrain model with best hyperparameters on concatenated X_train with X_val and evaluate using X_test

In [ ]:
# BOW
X1_tens_train_full = torch.cat((X1_tens_train, X1_tens_val), 0)
y1_tens_train_full = torch.cat((y1_tens_train, y1_tens_val), 0)

X2_tens_train_full = torch.cat((X2_tens_train, X2_tens_val), 0)
y2_tens_train_full = torch.cat((y2_tens_train, y2_tens_val), 0)

X3_tens_train_full = torch.cat((X3_tens_train, X3_tens_val), 0)
y3_tens_train_full = torch.cat((y3_tens_train, y3_tens_val), 0)

best_acc1 = model_train_best(model1, X1_tens_train_full, y1_tens_train_full, X1_tens_test, y1_tens_test, best_trial_bow1, get_vocab_size(X1_tens_train_full))
best_acc2 = model_train_best(model2, X2_tens_train_full, y2_tens_train_full, X2_tens_test, y2_tens_test, best_trial_bow2, get_vocab_size(X2_tens_train_full))
best_acc3 = model_train_best(model3, X3_tens_train_full, y3_tens_train_full, X3_tens_test, y3_tens_test, best_trial_bow3, get_vocab_size(X3_tens_train_full))

# print the metrics for the three datasets
print(f'Accuracy on df1 {best_acc1:.3f}')
print(f'Accuracy on df2 {best_acc2:.3f}')
print(f'Accuracy on df3 {best_acc3:.3f}')

Accuracy on df1 0.508
Accuracy on df2 0.460
Accuracy on df3 0.500


In [ ]:
# TFIDF
X1_tens_train_tfidf_full = torch.cat((X1_tens_train_tfidf, X1_tens_val_tfidf), 0)
y1_tens_train_tfidf_full = torch.cat((y1_tens_train_tfidf, y1_tens_val_tfidf), 0)

X2_tens_train_tfidf_full = torch.cat((X2_tens_train_tfidf, X2_tens_val_tfidf), 0)
y2_tens_train_tfidf_full = torch.cat((y2_tens_train_tfidf, y2_tens_val_tfidf), 0)

X3_tens_train_tfidf_full = torch.cat((X3_tens_train_tfidf, X3_tens_val_tfidf), 0)
y3_tens_train_tfidf_full = torch.cat((y3_tens_train_tfidf, y3_tens_val_tfidf), 0)

best_acc1_tfidf = model_train_best(model4, X1_tens_train_tfidf_full, y1_tens_train_tfidf_full, X1_tens_test_tfidf, y1_tens_test_tfidf, best_trial_tfidf1, get_vocab_size(X1_tens_train_tfidf_full))
best_acc2_tfidf = model_train_best(model5, X2_tens_train_tfidf_full, y2_tens_train_tfidf_full, X2_tens_test_tfidf, y2_tens_test_tfidf, best_trial_tfidf2, get_vocab_size(X2_tens_train_tfidf_full))
best_acc3_tfidf = model_train_best(model6, X3_tens_train_tfidf_full, y3_tens_train_tfidf_full, X3_tens_test_tfidf, y3_tens_test_tfidf, best_trial_tfidf3, get_vocab_size(X3_tens_train_tfidf_full))

# print the metrics for the three datasets
print(f'Accuracy on df1 {best_acc1_tfidf:.3f}')
print(f'Accuracy on df2 {best_acc2_tfidf:.3f}')
print(f'Accuracy on df3 {best_acc3_tfidf:.3f}')

Accuracy on df1 0.515
Accuracy on df2 0.498
Accuracy on df3 0.520


### Cross-validation

In [ ]:
# BOW 5-fold cross-validation test harness
kfold = StratifiedKFold(n_splits=5, shuffle=True)

def lr_cross_validate(model, X_train, y_train, best_trial, vocab_size):
  cv_scores = []
  for train, test in kfold.split(X_train, y_train):
    # create model, train, and get accuracy
    model = LogisticRegression(vocab_size, 1)
    acc = model_train_best(model, X_train[train], y_train[train], X_train[test], y_train[test], best_trial, vocab_size)
    print("Accuracy (LogReg): %.3f" % acc)
    cv_scores.append(acc)

  # evaluate the model
  acc = np.mean(cv_scores)
  std = np.std(cv_scores)
  # print("Model LogReg accuracy: %.2f%% (+/- %.2f%%)" % (acc2*100, std2*100))
  return print("Model LogReg accuracy: %.2f%% (+/- %.2f%%)" % (acc*100, std*100))



In [ ]:
#BOW CV
model11 = LogisticRegression(get_vocab_size(X1_bow_tens), 1)
df1_bow_lr_eval = lr_cross_validate(model11, X1_bow_tens, y1_bow_tens, best_trial_bow1, get_vocab_size(X1_bow_tens))

model22 = LogisticRegression(get_vocab_size(X2_bow_tens), 1)
df2_bow_lr_eval = lr_cross_validate(model22, X2_bow_tens, y2_bow_tens, best_trial_bow2, get_vocab_size(X2_bow_tens))

model33 = LogisticRegression(get_vocab_size(X3_bow_tens), 1)
df3_bow_lr_eval = lr_cross_validate(model33, X3_bow_tens, y3_bow_tens, best_trial_bow3, get_vocab_size(X3_bow_tens))

Accuracy (LogReg): 0.498
Accuracy (LogReg): 0.534
Accuracy (LogReg): 0.529
Accuracy (LogReg): 0.511
Accuracy (LogReg): 0.543
Model LogReg accuracy: 52.29% (+/- 1.64%)
Accuracy (LogReg): 0.483
Accuracy (LogReg): 0.514
Accuracy (LogReg): 0.484
Accuracy (LogReg): 0.512
Accuracy (LogReg): 0.516
Model LogReg accuracy: 50.17% (+/- 1.47%)
Accuracy (LogReg): 0.551
Accuracy (LogReg): 0.498
Accuracy (LogReg): 0.565
Accuracy (LogReg): 0.518
Accuracy (LogReg): 0.484
Model LogReg accuracy: 52.32% (+/- 3.07%)


In [ ]:
#TFID CV
model44 = LogisticRegression(get_vocab_size(X1_bow_tens), 1)
df1_tfidf_lr_eval = lr_cross_validate(model44, X1_tens_train_tfidf_full, y1_tens_train_tfidf_full, best_trial_tfidf1, get_vocab_size(X1_tens_train_tfidf_full))

model55 = LogisticRegression(get_vocab_size(X1_bow_tens), 1)
df2_tfidf_lr_eval = lr_cross_validate(model55, X2_tens_train_tfidf_full, y2_tens_train_tfidf_full, best_trial_tfidf2, get_vocab_size(X2_tens_train_tfidf_full))

model66 = LogisticRegression(get_vocab_size(X1_bow_tens), 1)
df3_tfidf_lr_eval = lr_cross_validate(model66, X3_tens_train_tfidf_full, y3_tens_train_tfidf_full, best_trial_tfidf3, get_vocab_size(X3_tens_train_tfidf_full))

Accuracy (LogReg): 0.477
Accuracy (LogReg): 0.518
Accuracy (LogReg): 0.533
Accuracy (LogReg): 0.495
Accuracy (LogReg): 0.507
Model LogReg accuracy: 50.60% (+/- 1.93%)
Accuracy (LogReg): 0.484
Accuracy (LogReg): 0.479
Accuracy (LogReg): 0.506
Accuracy (LogReg): 0.513
Accuracy (LogReg): 0.457
Model LogReg accuracy: 48.77% (+/- 2.00%)
Accuracy (LogReg): 0.522
Accuracy (LogReg): 0.499
Accuracy (LogReg): 0.486
Accuracy (LogReg): 0.486
Accuracy (LogReg): 0.475
Model LogReg accuracy: 49.34% (+/- 1.61%)


# Project demonstration
Prepare a summary of your work and discuss your choices with other course mates.

1. Prepare and carry out 5-10 min demo of your results and achievements in this project.
2. Publish your project on the GitHub or GitLab repository
3. Don't remember to add README file with description of this project, specification of your goals and assumptions and explanation of your approach and results.